In [1]:
import numpy as np
import pandas as pd
#import astropy.constants as const
import scipy.constants as const
import astropy.units as u
import argparse
import sys

In [5]:
def drop_time(h, g = 9.8): #h = positional, g = keyword
    t = ((2.0 * h) / g) ** (1/2) #Avoid using a package unless absolutely necessary
    return t

In [7]:
#If the height is 1 m:
drop_time(10)

1.4285714285714286

Machine Precision

In [1]:
x = 1.0
eps = 1.0

while not x + eps == x:
    eps = 0.5 * eps
    
print(2 * eps)

2.220446049250313e-16


In [7]:
b = 0.1
print(type(b))
print("{:30.20}".format(b))
sys.float_info

<class 'float'>
        0.10000000000000000555


sys.float_info(max=1.7976931348623157e+308, max_exp=1024, max_10_exp=308, min=2.2250738585072014e-308, min_exp=-1021, min_10_exp=-307, dig=15, mant_dig=53, epsilon=2.220446049250313e-16, radix=2, rounds=1)

In [9]:
x = 1
y = 1 + (10e-14 * np.sqrt(2))

print(1e14 * (y-x))
print(np.sqrt(2))

14.144241333724494
1.4142135623730951


In [10]:
def quad_formula_2(a, b, c):
    #Solution 2
    x_1 = (2 * c) / (-b - np.sqrt(np.square(b) - (4 * a * c)))
    x_2 = (2 * c) / (-b + np.sqrt(np.square(b) - (4 * a * c)))
    
    return x_1, x_2  

In [15]:
root_1, root_2 = quad_formula_2(3, 6, 1)
print(root_1)
print(root_2)

-0.18350341907227397
-1.8164965809277254


In [8]:
def energy(n):
    
    hf = 1
    beta = 0.01
    
    #Initializers
    Z = 0
    E = 0
    
    for i in range(n):
        E_n = hf * (i + 0.5) #Energy level
        Z = Z + np.exp(-beta * E_n) #Z equation (infinite sum)
        E += E_n * np.exp(-beta * E_n) #E equation
        
    avg_E = (E/Z) #Average energy
    
    return avg_E

In [9]:
#Hundred runs
print(f"4000 runs: {energy(4000)}")

4000 runs: 100.00083333194436


Trapezoid Rule (Integration)

In [3]:
def func(x): #Integration function
    return x**4 - (2*x) + 1

In [56]:
def trap_rule(n, a = 0, b = 2, function = func):
    
    #Default paramaters
    #a (lower limit) = 0
    #b (upper limit) = 2
    #function = func
    
    h = (b - a) / n #Height
    s = (0.5 * function(a)) + (0.5 * function(b)) #Sum
    
    for k in range(1, n + 1): #Summation (range stops at n)
        s += function(a + (k * h))
        
    return (h * s) #Multiply height by the sum

In [58]:
trap_rule(10)

7.106560000000001

Simpson's Rule

In [47]:
def simpsons_rule(n, a = 0, b = 2, function = func):
    
    #Default paramaters
    #a (lower limit) = 0
    #b (upper limit) = 2
    #function = func 
    
    delta_x = (b - a) / n
    
    s = function(a) + function(b)
    
    #Odd integer loop
    for k in range(1, int(n/2) + 1):
        s += 4 * function(a + (((2 * k) - 1) * delta_x))
    
    #Even integer loop
    for k in range(1, int((n/2))):
        s += 2 * function(a + (2 * k * delta_x))
        
    return (delta_x / 3) * s

In [48]:
simpsons_rule(10)

4.400426666666667

In [59]:
#Trapezoid rule

#10 steps
integral_10_trap = trap_rule(10)

#20 steps
integral_20_trap = trap_rule(20)

error_trap = (1/3) * abs(integral_20_trap - integral_10_trap)

print(f"Error with Trapezoid Rule: {error_trap}")

Error with Trapezoid Rule: 0.4599666666666664


In [49]:
#Simpson's rule

#10 steps
integral_10_simp = simpsons_rule(10)

#20 steps
integral_20_simp = simpsons_rule(20)

error_simp = (1/15) * abs(integral_20_simp - integral_10_simp)

print(f"Error with Simpson's Rule: {error_simp}")

Error with Simpson's Rule: 2.6666666666604518e-05


Gaussian Quadrature

In [6]:
def gaussxw(N):

    # Initial approximation to roots of the Legendre polynomial
    a = np.linspace(3,4*N-1,N)/(4*N+2)
    x = np.cos(np.pi*a+1/(8*N*N*np.tan(a)))

    # Find roots using Newton's method
    epsilon = 1e-15
    delta = 1.0
    while delta>epsilon:
        p0 = np.ones(N,float)
        p1 = np.copy(x)
        for k in range(1,N):
            p0,p1 = p1,((2*k+1)*x*p1-k*p0)/(k+1)
        dp = (N+1)*(p0-x*p1)/(1-x*x)
        dx = p1/dp
        x -= dx
        delta = np.max(np.abs(dx))

    # Calculate the weights
    w = 2*(N+1)*(N+1)/(N*N*(1-x*x)*dp*dp)

    return x,w

def gaussxwab(N,a,b):
    x,w = gaussxw(N)
    return 0.5*(b-a)*x+0.5*(b+a),0.5*(b-a)*w

In [17]:
locations, weights = gaussxwab(3, 0, 2)
print(f"Locations: {locations}\n")
print(f"Weights: {weights}")

Locations: [1.77459667 1.         0.22540333]

Weights: [0.55555556 0.88888889 0.55555556]


In [20]:
def gauss_quad(N, a = 0, b = 2, function = func): #N = Number of steps, a = Lower bound, b = Upper bound
    
    x, w = gaussxwab(N, a, b) #Calculates the x-coordinates for specific points and their weights
    
    s = 0 #Initializer variable for the sum
    
    for k in range(0, N): #Need the range to span the same indices as the x and w lists
        s += w[k] * function(x[k]) #Multiplies the weights by the y-values corresponding to each point's location
        
    return s #Returns the sum

In [23]:
gauss_quad(3)

4.4000000000000075